In [1]:
import pandas as pd
books=pd.read_csv("books_with_categories.csv")

In [3]:
from transformers import pipeline

classifier = pipeline(
    "text-classification",
    model="j-hartmann/emotion-english-distilroberta-base",
    top_k=None
)
classifier("I love this!")

Device set to use cpu


[[{'label': 'joy', 'score': 0.9771687984466553},
  {'label': 'surprise', 'score': 0.008528673090040684},
  {'label': 'neutral', 'score': 0.005764589179307222},
  {'label': 'anger', 'score': 0.004419779404997826},
  {'label': 'sadness', 'score': 0.002092391485348344},
  {'label': 'disgust', 'score': 0.0016119909705594182},
  {'label': 'fear', 'score': 0.00041385178337804973}]]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/329M [00:00<?, ?B/s]

In [4]:
books["description"][0]

'A NOVEL THAT READERS and critics have been eagerly anticipating for over a decade, Gilead is an astonishingly imagined story of remarkable lives. John Ames is a preacher, the son of a preacher and the grandson (both maternal and paternal) of preachers. It’s 1956 in Gilead, Iowa, towards the end of the Reverend Ames’s life, and he is absorbed in recording his family’s story, a legacy for the young son he will never see grow up. Haunted by his grandfather’s presence, John tells of the rift between his grandfather and his father: the elder, an angry visionary who fought for the abolitionist cause, and his son, an ardent pacifist. He is troubled, too, by his prodigal namesake, Jack (John Ames) Boughton, his best friend’s lost son who returns to Gilead searching for forgiveness and redemption. Told in John Ames’s joyous, rambling voice that finds beauty, humour and truth in the smallest of life’s details, Gilead is a song of celebration and acceptance of the best and the worst the world ha

In [5]:
classifier(books["description"][0])

[[{'label': 'fear', 'score': 0.6548418998718262},
  {'label': 'neutral', 'score': 0.16985182464122772},
  {'label': 'sadness', 'score': 0.11640854179859161},
  {'label': 'surprise', 'score': 0.020700592547655106},
  {'label': 'disgust', 'score': 0.01910073682665825},
  {'label': 'joy', 'score': 0.015161239542067051},
  {'label': 'anger', 'score': 0.00393515033647418}]]

In [6]:
classifier(books["description"][0].split("."))

[[{'label': 'surprise', 'score': 0.7296027541160583},
  {'label': 'neutral', 'score': 0.14038576185703278},
  {'label': 'fear', 'score': 0.06816209107637405},
  {'label': 'joy', 'score': 0.04794240742921829},
  {'label': 'anger', 'score': 0.00915635284036398},
  {'label': 'disgust', 'score': 0.0026284719351679087},
  {'label': 'sadness', 'score': 0.0021221607457846403}],
 [{'label': 'neutral', 'score': 0.449370414018631},
  {'label': 'disgust', 'score': 0.27359241247177124},
  {'label': 'joy', 'score': 0.10908260941505432},
  {'label': 'sadness', 'score': 0.09362703561782837},
  {'label': 'anger', 'score': 0.040478307753801346},
  {'label': 'surprise', 'score': 0.02697017230093479},
  {'label': 'fear', 'score': 0.006879065651446581}],
 [{'label': 'neutral', 'score': 0.6462168097496033},
  {'label': 'sadness', 'score': 0.2427326887845993},
  {'label': 'disgust', 'score': 0.04342268034815788},
  {'label': 'surprise', 'score': 0.028300466015934944},
  {'label': 'joy', 'score': 0.014211482

In [7]:
sentences=books["description"][0].split(".")
predictions=classifier(sentences)

In [8]:
sentences[0]

'A NOVEL THAT READERS and critics have been eagerly anticipating for over a decade, Gilead is an astonishingly imagined story of remarkable lives'

In [9]:
predictions[0]

[{'label': 'surprise', 'score': 0.7296027541160583},
 {'label': 'neutral', 'score': 0.14038576185703278},
 {'label': 'fear', 'score': 0.06816209107637405},
 {'label': 'joy', 'score': 0.04794240742921829},
 {'label': 'anger', 'score': 0.00915635284036398},
 {'label': 'disgust', 'score': 0.0026284719351679087},
 {'label': 'sadness', 'score': 0.0021221607457846403}]

In [13]:
sorted(predictions[0], key=lambda x:x["label"])

[{'label': 'anger', 'score': 0.00915635284036398},
 {'label': 'disgust', 'score': 0.0026284719351679087},
 {'label': 'fear', 'score': 0.06816209107637405},
 {'label': 'joy', 'score': 0.04794240742921829},
 {'label': 'neutral', 'score': 0.14038576185703278},
 {'label': 'sadness', 'score': 0.0021221607457846403},
 {'label': 'surprise', 'score': 0.7296027541160583}]

In [19]:
import numpy as np
emotion_labels=['anger','disgust','fear', 'joy','sadness','surprise', 'neutral']
isbn=[]
emotion_scores={label: []for label in emotion_labels}

def calculate_max_emotion_scores(predictions):
    per_emotion_scores={label: []for label in emotion_labels}
    for prediction in predictions:
        sorted_predictions=sorted(prediction,key=lambda x:x["label"])
        for index,label in enumerate(emotion_labels):
            per_emotion_scores[label].append(sorted_predictions[index]["score"])
        return{label: np.max(scores)for label,scores in per_emotion_scores.items()}

In [20]:
for i in range(10):
    isbn.append(books["isbn13"][i])
    sentences=books["description"][i].split(".")
    predictions=classifier(sentences)
    max_scores=calculate_max_emotion_scores(predictions)
    for label in emotion_labels:
        emotion_scores[label].append(max_scores[label])

In [21]:
emotion_scores

{'anger': [np.float64(0.00915635284036398),
  np.float64(0.005966844502836466),
  np.float64(0.04130087420344353),
  np.float64(0.016036177054047585),
  np.float64(0.013624361716210842),
  np.float64(0.009158502332866192),
  np.float64(0.003973888698965311),
  np.float64(0.023656237870454788),
  np.float64(0.30066895484924316),
  np.float64(0.01765836775302887)],
 'disgust': [np.float64(0.0026284719351679087),
  np.float64(0.0028870662208646536),
  np.float64(0.024568378925323486),
  np.float64(0.06069502979516983),
  np.float64(0.12224263697862625),
  np.float64(0.012228705920279026),
  np.float64(0.003816465148702264),
  np.float64(0.009091978892683983),
  np.float64(0.27948135137557983),
  np.float64(0.17792679369449615)],
 'fear': [np.float64(0.06816209107637405),
  np.float64(0.0038098806980997324),
  np.float64(0.10406126081943512),
  np.float64(0.0016916330205276608),
  np.float64(0.0950433686375618),
  np.float64(0.03679516911506653),
  np.float64(0.0012676806654781103),
  np.f

In [23]:
from tqdm import tqdm

emotion_labels=['anger','disgust','fear', 'joy','sadness','surprise', 'neutral']
isbn=[]
emotion_scores={label: []for label in emotion_labels}

for i in tqdm(range(len(books))):
    isbn.append(books["isbn13"][i])
    sentences=books["description"][i].split(".")
    predictions=classifier(sentences)
    max_scores=calculate_max_emotion_scores(predictions)
    for label in emotion_labels:
        emotion_scores[label].append(max_scores[label])

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 5197/5197 [14:05<00:00,  6.15it/s]


In [27]:
emotions_df=pd.DataFrame(emotion_scores)
emotions_df["isbn13"]=isbn

In [28]:
emotions_df.head()

,anger,disgust,fear,joy,sadness,surprise,neutral,isbn13
0,0.009156,0.002628,0.068162,0.047942,0.140386,0.002122,0.729603,9780002005883
1,0.005967,0.002887,0.003810,0.704422,0.217760,0.004509,0.060646,9780002261982
2,0.041301,0.024568,0.104061,0.767237,0.042176,0.010860,0.009796,9780006178736
3,0.016036,0.060695,0.001692,0.161757,0.732685,0.020988,0.006147,9780006280897
4,0.013624,0.122243,0.095043,0.008336,0.272614,0.475881,0.012260,9780006280934


In [31]:
books=pd.merge(books,emotions_df, on="isbn13")

In [32]:
books

,isbn13,isbn10,title,authors,categories,thumbnail,description,published_year,average_rating,num_pages,...,title_&_subtitle,tagged_description,simple_categories,anger,disgust,fear,joy,sadness,surprise,neutral
0,9780002005883,0002005883,Gilead,Marilynne Robinson,Fiction,http://books.google.com/books/content?id=KQZCP...,A NOVEL THAT READERS and critics have been eag...,2004.0,3.85,247.0,...,Gilead,9780002005883; A NOVEL THAT READERS and critic...,Fiction,0.009156,0.002628,0.068162,0.047942,0.140386,0.002122,0.729603
1,9780002261982,0002261987,Spider's Web,Charles Osborne;Agatha Christie,Detective and mystery stories,http://books.google.com/books/content?id=gA5GP...,A new 'Christie for Christmas' -- a full-lengt...,2000.0,3.83,241.0,...,Spider's Web; A Novel,9780002261982; A new 'Christie for Christmas' ...,Fiction,0.005967,0.002887,0.003810,0.704422,0.217760,0.004509,0.060646
2,9780006178736,0006178731,Rage of angels,Sidney Sheldon,Fiction,http://books.google.com/books/content?id=FKo2T...,"A memorable, mesmerizing heroine Jennifer -- b...",1993.0,3.93,512.0,...,Rage of angels,"9780006178736; A memorable, mesmerizing heroin...",Fiction,0.041301,0.024568,0.104061,0.767237,0.042176,0.010860,0.009796
3,9780006280897,0006280897,The Four Loves,Clive Staples Lewis,Christian life,http://books.google.com/books/content?id=XhQ5X...,Lewis' work on the nature of love divides love...,2002.0,4.15,170.0,...,The Four Loves,9780006280897; Lewis' work on the nature of lo...,Nonfiction,0.016036,0.060695,0.001692,0.161757,0.732685,0.020988,0.006147
4,9780006280934,0006280935,The Problem of Pain,Clive Staples Lewis,Christian life,http://books.google.com/books/content?id=Kk-uV...,"""In The Problem of Pain, C.S. Lewis, one of th...",2002.0,4.09,176.0,...,The Problem of Pain,"9780006280934; ""In The Problem of Pain, C.S. L...",Nonfiction,0.013624,0.122243,0.095043,0.008336,0.272614,0.475881,0.012260
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5192,9788172235222,8172235224,Mistaken Identity,Nayantara Sahgal,Indic fiction (English),http://books.google.com/books/content?id=q-tKP...,On A Train Journey Home To North India After L...,2003.0,2.93,324.0,...,Mistaken Identity,9788172235222; On A Train Journey Home To Nort...,Fiction,0.025156,0.001939,0.094667,0.002254,0.010511,0.857255,0.008218
5193,9788173031014,8173031010,Journey to the East,Hermann Hesse,Adventure stories,http://books.google.com/books/content?id=rq6JP...,This book tells the tale of a man who goes on ...,2002.0,3.70,175.0,...,Journey to the East,9788173031014; This book tells the tale of a m...,Nonfiction,0.005602,0.003775,0.018216,0.400263,0.338892,0.005487,0.227765
5194,9788179921623,817992162X,The Monk Who Sold His Ferrari: A Fable About F...,Robin Sharma,Health & Fitness,http://books.google.com/books/content?id=c_7mf...,"Wisdom to Create a Life of Passion, Purpose, a...",2003.0,3.82,198.0,...,The Monk Who Sold His Ferrari: A Fable About F...,9788179921623; Wisdom to Create a Life of Pass...,Fiction,0.008463,0.009147,0.013295,0.620453,0.329753,0.010788,0.008101
5195,9788185300535,8185300534,I Am that,Sri Nisargadatta Maharaj;Sudhakar S. Dikshit,Philosophy,http://books.google.com/books/content?id=Fv_JP...,This collection of the timeless teachings of o...,1999.0,4.51,531.0,...,I Am that; Talks with Sri Nisargadatta Maharaj,9788185300535; This collection of the timeless...,Nonfiction,0.005475,0.034545,0.003970,0.258353,0.648011,0.017372,0.032275


In [33]:
books.to_csv("books_with_emotions.csv",index=False)